In [ ]:
# Clone the repos
!git clone https://github.com/metrica-sports/sample-data.git
!git clone https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking.git
# I took too long to work on this, and now the pitch control script has been
# updated :) Revert back to the original commit hash for now...
!cd LaurieOnTracking && git reset --hard e047ede88e11030a9755ee783614f9c960664c01

In [ ]:
# Some of the Metrica_* scripts expect the script directory to be the working
# directory, so we need to add it as an environment path in order to import them
# in this notebook.
# https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.insert(1, '/content/LaurieOnTracking')

from collections import defaultdict
import concurrent.futures
import datetime
from IPython.display import clear_output
from ipywidgets import HTML
import json
from LaurieOnTracking import Metrica_IO as mio
from LaurieOnTracking import Metrica_PitchControl as mpc
from LaurieOnTracking import Metrica_Velocities as mvelo
from LaurieOnTracking import Metrica_Viz as mviz
import matplotlib.animation as anim
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from typing import Dict, List, Set, Tuple

In [ ]:
%load_ext Cython

In [ ]:
%%cython
cimport cython
from libc.math cimport exp, log, M_PI, pow, sqrt
import numpy as np
cimport numpy as np

################################################################################
# Constant params. Most of these are stored in a Python dictionary.
# https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/e047ede88e11030a9755ee783614f9c960664c01/Metrica_PitchControl.py#L143-L161
################################################################################
cdef:
  # Maximum player acceleration in m/s^2
  double MAX_PLAYER_ACCEL = 7.0
  # Max player speed in m/s
  double MAX_PLAYER_SPEED = 5.0
  # Player reaction time in seconds
  double REACTION_TIME = 0.7
  # Average ball travel speed in m/s
  double AVG_BALL_SPEED = 15.0
  # Upper limit on itegral time
  double MAX_INTGL_TIME = 10.0
  # Integration timestep (dt)
  double INTGL_DT = 0.04
  # Assume convergence when PPCF>0.99 at a given location
  double MODEL_CONVERGE_TOL = 0.01
  # Standard diviation of sigmoid, determines uncertainty in player arrival time
  double TTI_SIGMA = 0.45
  # Ball ctrl for constant attacking team
  double LAMBDA_ATT = 4.3
  # Ball ctrl for constant defending team
  double LAMBDA_DEF = 4.3
  # Coefficient to determine when to skip the pitch control calculation
  double TIME_TO_CTRL_VETO = 3.0
  double TIME_TO_CTRL_ATT = time_until_ctrl_override_for_team_cy(LAMBDA_ATT)
  double TIME_TO_CTRL_DEF = time_until_ctrl_override_for_team_cy(LAMBDA_DEF)

################################################################################
# This is calculated once in default_model_params()
# https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/e047ede88e11030a9755ee783614f9c960664c01/Metrica_PitchControl.py#L159-L160
################################################################################
@cython.cdivision(True)
cdef double time_until_ctrl_override_for_team_cy(double team_lambda):
  return (TIME_TO_CTRL_VETO * log(10.0) *
          (sqrt(3.0) * TTI_SIGMA / M_PI + 1 / team_lambda))

################################################################################
# In the Metrica_PitchControl.py script, this is a class function on the
# player() class. Since Python classes don't translate well to Cython, this
# function calculates time-to-intercept on demand given a player's
# coordinates.
# https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/e047ede88e11030a9755ee783614f9c960664c01/Metrica_PitchControl.py#L110-L116
################################################################################
@cython.cdivision(True)
cdef double simple_time_to_intercept_cy(double position_x,
                                        double position_y,
                                        double velocity_x,
                                        double velocity_y,
                                        double final_pos_x,
                                        double final_pos_y):
  cdef:
    double tti
    double norm_x
    double norm_y
    double norm
    double sum_of_sqrs
  cdef double norm_arr[2]
  r_reaction_x = position_x + velocity_x * REACTION_TIME
  r_reaction_y = position_y + velocity_y * REACTION_TIME
  norm_x = final_pos_x - r_reaction_x
  norm_y = final_pos_y - r_reaction_y
  # The next 2 lines re-create np.linalg.norm()
  sum_of_sqrs = pow(norm_x, 2) + pow(norm_y, 2)
  norm = sqrt(sum_of_sqrs)
  tti = REACTION_TIME + norm / MAX_PLAYER_SPEED
  return tti

################################################################################
# This is also a class function on the player() class, but here it is
# calculated on demand using pure C code.
# https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/e047ede88e11030a9755ee783614f9c960664c01/Metrica_PitchControl.py#L118-L121
################################################################################
@cython.cdivision(True)
cdef double prob_intercept_cy(double time_t, double time_to_intercept):
  cdef double prob
  prob = 1.0 / (1.0 + exp(-1.0 * M_PI / sqrt(3.0) / TTI_SIGMA *
                          (time_t - time_to_intercept)))
  return prob

################################################################################
# This is done with a list comprehension in Python, a stores the time-to-
# intercept as a class attribute on each player. Here we store each player's
# time-to-intercept in an array and return the calculated min time-to-
# intercept.
# https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/e047ede88e11030a9755ee783614f9c960664c01/Metrica_PitchControl.py#L244-L245
################################################################################
@cython.boundscheck(False)
@cython.wraparound(False)
cdef double calculate_min_time_to_intercept_cy(list players,
                                               double target_pos_x,
                                               double target_pos_y,
                                               double[:] tti_arr):
  cdef double tau_min_att = MAX_INTGL_TIME  # All times should be less than this
  cdef double simp_tti_att
  cdef size_t idx = 0
  for player in players:
    simp_tti_att = simple_time_to_intercept_cy(player.position[0],
                                               player.position[1],
                                               player.velocity[0],
                                               player.velocity[1],
                                               target_pos_x,
                                               target_pos_y)
    tti_arr[idx] = simp_tti_att
    if simp_tti_att < tau_min_att:
      tau_min_att = simp_tti_att
    idx += 1
  return tau_min_att

################################################################################
# The main pitch control calculation. Most of it follows the Python code, but
# is more verbose because of C. There is still some reliance on Numpy which
# could be converted to C with some effort.
# https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/e047ede88e11030a9755ee783614f9c960664c01/Metrica_PitchControl.py#L217
################################################################################
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
def calculate_pitch_control_at_target_cy(double target_pos_x,
                                         double target_pos_y,
                                         list attacking_players,
                                         list defending_players,
                                         int num_att_players,
                                         int num_def_players,
                                         double ball_start_pos_x,
                                         double ball_start_pos_y):
  cdef:
    double sum_of_sqrs,
    double norm,
    double ball_travel_time
  cdef double ball_travel_time_norm_arr[2]
  # Calculate ball travel time
  ball_travel_time_norm_arr[0] = target_pos_x - ball_start_pos_x
  ball_travel_time_norm_arr[1] = target_pos_y - ball_start_pos_y
  sum_of_sqrs = 0.0
  for i in range(2):
    sum_of_sqrs += pow(ball_travel_time_norm_arr[i], 2)
  norm = sqrt(sum_of_sqrs)
  ball_travel_time = norm / AVG_BALL_SPEED
  # Calculate arrival times for attacking team
  cdef double player_ttis_att[11]
  cdef double min_tti_att
  min_tti_att = calculate_min_time_to_intercept_cy(attacking_players,
                                                   target_pos_x,
                                                   target_pos_y,
                                                   player_ttis_att)
  # Calculate arrival times for defending team
  cdef double player_ttis_def[11]
  cdef double min_tti_def
  min_tti_def = calculate_min_time_to_intercept_cy(defending_players,
                                                   target_pos_x,
                                                   target_pos_y,
                                                   player_ttis_def)
  cdef double closer_obj_att
  cdef double closer_obj_def
  cdef double[:] dt_array
  cdef double[:] ppcf_att
  cdef double[:] ppcf_def
  cdef double ptot = 0.0
  cdef int idx = 1
  cdef double T
  cdef double player_tti
  cdef double d_ppcf_dt = 0.0
  cdef double d_ppcf_dt_intgl
  cdef Py_ssize_t player_tti_idx = 1
  cdef double player_ppcf_att[11]
  cdef double player_ppcf_def[11]
  cdef Py_ssize_t init_idx = 0
  for init_idx in range(11):
    player_ppcf_att[init_idx] = 0.0
    player_ppcf_def[init_idx] = 0.0
  if ball_travel_time > min_tti_def:
    closer_obj_att = ball_travel_time
  else:
    closer_obj_att = min_tti_def
  if ball_travel_time > min_tti_att:
    closer_obj_def = ball_travel_time
  else:
    closer_obj_def = min_tti_att
  if (min_tti_att - closer_obj_att) >= TIME_TO_CTRL_DEF:
    return 0.0, 1.0
  elif (min_tti_def - closer_obj_def) >= TIME_TO_CTRL_ATT:
    return 1.0, 0.0
  else:
    dt_array = np.arange(ball_travel_time - INTGL_DT,
                         ball_travel_time + MAX_INTGL_TIME,
                         INTGL_DT)
    ppcf_att = np.zeros_like(dt_array)
    ppcf_def = np.zeros_like(dt_array)
    while 1.0 - ptot > MODEL_CONVERGE_TOL and idx < dt_array.shape[0]:
      T = dt_array[idx]
      for player_tti_idx in range(num_att_players):
      # for player in attacking_players:
        # player_tti_idx += 1
        player_tti = player_ttis_att[player_tti_idx]
        if player_tti - min_tti_att > TIME_TO_CTRL_ATT:
          continue
        d_ppcf_dt = ((1.0 - ppcf_att[idx - 1] - ppcf_def[idx - 1]) *
                     prob_intercept_cy(T, player_tti) * LAMBDA_ATT)
        # assert d_ppcf_dt > 0
        player_ppcf_att[player_tti_idx + 1] += d_ppcf_dt * INTGL_DT
        ppcf_att[idx] += player_ppcf_att[player_tti_idx + 1]
      for player_tti_idx in range(num_def_players):
      # for player in defending_players:
        # player_tti_idx += 1
        player_tti = player_ttis_def[player_tti_idx]
        if player_tti - min_tti_def > TIME_TO_CTRL_DEF:
          continue
        d_ppcf_dt = ((1.0 - ppcf_att[idx - 1] - ppcf_def[idx - 1]) *
                     prob_intercept_cy(T, player_tti) * LAMBDA_DEF)
        # assert d_ppcf_dt > 0
        player_ppcf_def[player_tti_idx + 1] += d_ppcf_dt * INTGL_DT
        ppcf_def[idx] += player_ppcf_def[player_tti_idx + 1]
      ptot = ppcf_def[idx] + ppcf_att[idx]
      idx += 1
    if idx > dt_array.size:
      print('Integration failed to converge')
    return ppcf_att[idx - 1], ppcf_def[idx - 1]

################################################################################
# This function is the entry point from Python code. It creates the X and Y
# grids for the pitch regions and returns the Numpy arrays of pitch contrl for
# each team, similar to the generate_pitch_control_for_event() function in
# Python.
# https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/e047ede88e11030a9755ee783614f9c960664c01/Metrica_PitchControl.py#L190-L215
################################################################################
@cython.cdivision(True)
def calculate_pitch_control_cy(double pitch_dimen_x,
                               double pitch_dimen_y,
                               int n_grid_cells_x,
                               int n_grid_cells_y,
                               list home_players,
                               list away_players,
                               double ball_start_pos_x,
                               double ball_start_pos_y):
  cdef Py_ssize_t x, y
  cdef double pitch_dimen_x_half, pitch_dimen_y_half
  cdef np.ndarray[np.float64_t, ndim=2] pitch_ctrl_a_cy
  cdef np.ndarray[np.float64_t, ndim=2] pitch_ctrl_d_cy
  cdef np.ndarray[np.float64_t, ndim=1] xgrid, ygrid, target_pos
  xgrid = np.linspace(-pitch_dimen_x/2.0, pitch_dimen_x/2.0, n_grid_cells_x)
  ygrid = np.linspace(-pitch_dimen_y/2.0, pitch_dimen_y/2.0, n_grid_cells_y)
  pitch_ctrl_a_cy = np.zeros(shape=(n_grid_cells_y, n_grid_cells_x))
  pitch_ctrl_d_cy = np.zeros(shape=(n_grid_cells_y, n_grid_cells_x))
  for y in range(n_grid_cells_y):
    for x in range(n_grid_cells_x):
      target_pos = np.array([xgrid[x], ygrid[y]])
      pitch_ctrl_a_cy[y, x], pitch_ctrl_d_cy[y, x] = \
          calculate_pitch_control_at_target_cy(target_pos[0],
                                               target_pos[1],
                                               home_players,
                                               away_players,
                                               len(home_players),
                                               len(away_players),
                                               ball_start_pos_x,
                                               ball_start_pos_y)
  return pitch_ctrl_a_cy, pitch_ctrl_d_cy

In [ ]:
# Looking at match #2 in the sample data directory
DATA_DIR = '/content/sample-data/data'
MATCH_ID = 2

def populate_tracking_dataframes() -> Tuple[pd.DataFrame, pd.DataFrame,
                                            pd.DataFrame]:
  """Creates DataFrames from the imported data."""
  tracking_home = mio.tracking_data(DATA_DIR, MATCH_ID, 'Home')
  tracking_away = mio.tracking_data(DATA_DIR, MATCH_ID, 'Away')
  events = mio.read_event_data(DATA_DIR, MATCH_ID)

  tracking_home = mio.to_metric_coordinates(tracking_home)
  tracking_away = mio.to_metric_coordinates(tracking_away)
  events = mio.to_metric_coordinates(events)

  tracking_home = mvelo.calc_player_velocities(tracking_home, smoothing=True)
  tracking_away = mvelo.calc_player_velocities(tracking_away, smoothing=True)

  return mio.to_single_playing_direction(tracking_home, tracking_away, events)

# Populate some DataFrames with Metrica data.
tracking_home, tracking_away, events = populate_tracking_dataframes()
shots = events[events['Type']=='SHOT']
goals = shots[shots['Subtype'].str.contains('-GOAL')].copy()

In [ ]:
def create_player_scenarios(team_df: pd.DataFrame, player_numbers: List[int],
                            team: str) -> Dict[str, pd.DataFrame]:
  """Creates a DataFrame for each player where the player does not move.

  In order to calculate the effect of a player on overall team pitch control,
  we create a scenario for each player where they do not move during the entire
  event sequence. This is simulated by setting the values to be the same for
  each frame of the DataFrame for these columns:
  - <team>_<player number>_x (player x position)
  - <team>_<player number>_y (player y position)
  - <team>_<player number>_vx (player x velocity)
  - <team>_<player number>_vy (player y velocity)
  - <team>_<player number>_speed (player speed)

  Returns:
    A dict where:
      keys are strings in the form of "Player_<player number>".
      values are DataFrames with the given player set to stationary.
  """
  player_scenarios = {}
  for player_number in player_numbers:
    team_without_current_player = team_df.copy()
    for col in ['{}_{}_x', '{}_{}_y', '{}_{}_vx', '{}_{}_vy', '{}_{}_speed']:
      col_name = col.format(team, player_number)
      team_without_current_player[col_name].values[:] = \
          team_without_current_player[col_name].iloc(0)[0]
    player_scenarios['Player_{}'.format(player_number)] = \
        team_without_current_player
  return player_scenarios


In [ ]:
# Use this cell to initialize all variables we need.
params = mpc.default_model_params(3)
field_dimen = (106.0, 68.0)
n_grid_cells_x = 50
n_grid_cells_y = int(n_grid_cells_x * field_dimen[1] / field_dimen[0])

# Event ID of the last goal
event_id = goals.index[-1]
start_frame = events.loc[event_id - 4]['Start Frame']
# Add a few post-goal frames
end_frame = events.loc[event_id]['End Frame'] + 10

home_team = tracking_home.loc[start_frame:end_frame]
away_team = tracking_away.loc[start_frame:end_frame]
attacking_side = events.loc[event_id].Team

# We will keep track of the affect each player has on the overall team pitch
# control by calculating what the team pitch control would have been if the
# player had not moved from their initial position over the duration of the
# event sequence.
player_pitch_control = defaultdict(list)

# Extract the player numbers who are active during this event sequence for the
# given team. Column names are in the form of: "Home_2_x" or "Away_10_speed",
# etc. If the column contains null, it means the player is not active.
def get_player_numbers(team_df: pd.DataFrame, team_name: str) -> Set[str]:
  """Returns the set of player numbers, as strings, for the given team."""
  return set([colname.split('_')[1]
              for colname in team_df.columns
              if team_name in colname
              and not team_df[colname].isnull().any()])

if attacking_side == 'Home':
  attacking_team = home_team[:]
  defending_team = away_team[:]
  defending_side = 'Away'
else:
  attacking_team = away_team[:]
  defending_team = home_team[:]
  defending_side = 'Home'

attacking_player_numbers = get_player_numbers(attacking_team,
                                              attacking_side)
attacking_player_scenarios = create_player_scenarios(attacking_team,
                                                     attacking_player_numbers,
                                                     attacking_side)
defending_player_numbers = get_player_numbers(defending_team,
                                              defending_side)
defending_player_scenarios = create_player_scenarios(defending_team,
                                                     defending_player_numbers,
                                                     defending_side)

The following cell will use the Cython code to generate pitch control percentages for each team and player for each frame, and print out runtime statistics.

In [ ]:
# Keep the scope of ball position outside of the main loop so the last ball
# position value can be used when the current value is NaN (after a goal score).
ball_start_pos = None
frame_range = home_team.index
end_frame = frame_range[0] + len(frame_range) - 1
# Keep track of the running time for each loop
time_history = []
start_time = time.time()
for frame_id in frame_range:
  clear_output(wait=True)
  print(f'Currently processing frame {frame_id} (until frame {end_frame})')
  loop_time = time.time() - start_time
  print('Last loop: {:.2f} secs.'.format(loop_time))
  time_history.append(loop_time)
  # The time of the first loop always skews the average, so skip it.
  if len(time_history) > 1:
    secs_left = np.mean(time_history[1:]) * (end_frame + 1 - frame_id)
    print(f'Estimated time left: {datetime.timedelta(seconds=secs_left)}')
  start_time = time.time()
  # Full team players for both sides from the tracking data.
  attacking_players = mpc.initialise_players(attacking_team.loc[frame_id],
                                             attacking_side,
                                             params)
  defending_players = mpc.initialise_players(defending_team.loc[frame_id],
                                             defending_side,
                                             params)
  # After the goal score, the ball position is NaN.
  if not np.isnan(home_team.loc[frame_id]['ball_x']):
    ball_start_pos = np.array([home_team.loc[frame_id]['ball_x'],
                               home_team.loc[frame_id]['ball_y']])
  # Use a process pool to distribute the work.
  fn_futures = {}
  num_workers = max(len(attacking_player_numbers),
                    len(defending_player_numbers))
  with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as exe:
    fn_futures[exe.submit(calculate_pitch_control_cy,
                          field_dimen[0],
                          field_dimen[1],
                          n_grid_cells_x,
                          n_grid_cells_y,
                          attacking_players,
                          defending_players,
                          ball_start_pos[0],
                          ball_start_pos[1])] = 'Team'
    for player_label, player_scenario in attacking_player_scenarios.items():
      attacking_scenario = mpc.initialise_players(player_scenario.loc[frame_id],
                                                  attacking_side,
                                                  params)
      fn_futures[exe.submit(calculate_pitch_control_cy,
                             field_dimen[0],
                             field_dimen[1],
                             n_grid_cells_x,
                             n_grid_cells_y,
                             attacking_scenario,
                             defending_players,
                             ball_start_pos[0],
                             ball_start_pos[1])] = 'att_' + player_label
    for player_label, player_scenario in defending_player_scenarios.items():
      defending_scenario = mpc.initialise_players(player_scenario.loc[frame_id],
                                                  defending_side,
                                                  params)
      fn_futures[exe.submit(calculate_pitch_control_cy,
                            field_dimen[0],
                            field_dimen[1],
                            n_grid_cells_x,
                            n_grid_cells_y,
                            attacking_players,
                            defending_scenario,
                            ball_start_pos[0],
                            ball_start_pos[1])] = 'def_' + player_label
    for fn_future in concurrent.futures.as_completed(fn_futures):
      pitch_ctrl = fn_future.result()
      player_pitch_control[fn_futures[fn_future]].append(pitch_ctrl)
# Print all runtime statistics.
print('Done')
print('Avg loop time: {:.2f} secs.'.format(np.mean(time_history[1:])))
print('Min loop time: {:.2f} secs.'.format(sorted(time_history)[1]))
print('Max loop time: {:.2f} secs.'.format(max(time_history)))
print(f'Total running time: {datetime.timedelta(seconds=sum(time_history))}')

Currently processing frame 121065 (until frame 121065)
Last loop: 1.53 secs.
Estimated time left: 0:00:01.535264
Done
Avg loop time: 1.54 secs.
Min loop time: 1.46 secs.
Max loop time: 1.62 secs.
Total running time: 0:05:43.901274


In [ ]:
#@title Custom plot_pitch2 implementation (code is hidden)
# Need to make an adjustment to the Metrica_Viz.plot_pitch() function
# to provide the matplotlib figure and axes so that the pitch can be
# redrawn each time we record a frame for the output video.
def plot_pitch2(figax=None, field_dimen = (106.0,68.0), field_color ='green', linewidth=2,
                markersize=20):
    """ plot_pitch2

    Plots a soccer pitch. All distance units converted to meters.

    Parameters
    -----------
        fig,ax     : Can be used to pass in the (fig,ax) objects of a previously generated pitch. Set to (fig,ax) to use an existing figure, or None (the default) to generate a new pitch plot,
        field_dimen: (length, width) of field in meters. Default is (106,68)
        field_color: color of field. options are {'green','white'}
        linewidth  : width of lines. default = 2
        markersize : size of markers (e.g. penalty spot, centre spot, posts). default = 20

    Returrns
    -----------
       fig,ax : figure and aixs objects (so that other data can be plotted onto the pitch)
    """
    # Custom code to use a figure and axis if provided.
    if figax is None:
      fig,ax = plt.subplots(figsize=(12,8)) # create a figure
    else:
      fig, ax = figax
    # decide what color we want the field to be. Default is green, but can also choose white
    if field_color=='green':
        ax.set_facecolor('mediumseagreen')
        lc = 'whitesmoke' # line color
        pc = 'w' # 'spot' colors
    elif field_color=='white':
        lc = 'k'
        pc = 'k'
    # ALL DIMENSIONS IN m
    border_dimen = (3,3) # include a border arround of the field of width 3m
    meters_per_yard = 0.9144 # unit conversion from yards to meters
    half_pitch_length = field_dimen[0]/2. # length of half pitch
    half_pitch_width = field_dimen[1]/2. # width of half pitch
    signs = [-1,1]
    # Soccer field dimensions typically defined in yards, so we need to convert to meters
    goal_line_width = 8*meters_per_yard
    box_width = 20*meters_per_yard
    box_length = 6*meters_per_yard
    area_width = 44*meters_per_yard
    area_length = 18*meters_per_yard
    penalty_spot = 12*meters_per_yard
    corner_radius = 1*meters_per_yard
    D_length = 8*meters_per_yard
    D_radius = 10*meters_per_yard
    D_pos = 12*meters_per_yard
    centre_circle_radius = 10*meters_per_yard
    # plot half way line # center circle
    ax.plot([0,0],[-half_pitch_width,half_pitch_width],lc,linewidth=linewidth)
    ax.scatter(0.0,0.0,marker='o',facecolor=lc,linewidth=0,s=markersize)
    y = np.linspace(-1,1,50)*centre_circle_radius
    x = np.sqrt(centre_circle_radius**2-y**2)
    ax.plot(x,y,lc,linewidth=linewidth)
    ax.plot(-x,y,lc,linewidth=linewidth)
    for s in signs: # plots each line seperately
        # plot pitch boundary
        ax.plot([-half_pitch_length,half_pitch_length],[s*half_pitch_width,s*half_pitch_width],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length],[-half_pitch_width,half_pitch_width],lc,linewidth=linewidth)
        # goal posts & line
        ax.plot( [s*half_pitch_length,s*half_pitch_length],[-goal_line_width/2.,goal_line_width/2.],pc+'s',markersize=6*markersize/20.,linewidth=linewidth)
        # 6 yard box
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*box_length],[box_width/2.,box_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*box_length],[-box_width/2.,-box_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length-s*box_length,s*half_pitch_length-s*box_length],[-box_width/2.,box_width/2.],lc,linewidth=linewidth)
        # penalty area
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*area_length],[area_width/2.,area_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length,s*half_pitch_length-s*area_length],[-area_width/2.,-area_width/2.],lc,linewidth=linewidth)
        ax.plot([s*half_pitch_length-s*area_length,s*half_pitch_length-s*area_length],[-area_width/2.,area_width/2.],lc,linewidth=linewidth)
        # penalty spot
        ax.scatter(s*half_pitch_length-s*penalty_spot,0.0,marker='o',facecolor=lc,linewidth=0,s=markersize)
        # corner flags
        y = np.linspace(0,1,50)*corner_radius
        x = np.sqrt(corner_radius**2-y**2)
        ax.plot(s*half_pitch_length-s*x,-half_pitch_width+y,lc,linewidth=linewidth)
        ax.plot(s*half_pitch_length-s*x,half_pitch_width-y,lc,linewidth=linewidth)
        # draw the D
        y = np.linspace(-1,1,50)*D_length # D_length is the chord of the circle that defines the D
        x = np.sqrt(D_radius**2-y**2)+D_pos
        ax.plot(s*half_pitch_length-s*x,y,lc,linewidth=linewidth)

    # remove axis labels and ticks
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    # set axis limits
    xmax = field_dimen[0]/2. + border_dimen[0]
    ymax = field_dimen[1]/2. + border_dimen[1]
    ax.set_xlim([-xmax,xmax])
    ax.set_ylim([-ymax,ymax])
    ax.set_axisbelow(True)
    return fig,ax

The following cell will plot all the pitch control data for each frame and save it to a video file.

In [ ]:
%%capture
# Create a horizontal layout with 3 subplots
plt.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(24, 12))
gs_att, gs_pitch, gs_def = gridspec.GridSpec(1, 3,
                                             width_ratios=[1.5, 8.0, 1.5],
                                             wspace=0.01)
ax_att = fig.add_subplot(gs_att)
ax_pitch = fig.add_subplot(gs_pitch)
ax_def = fig.add_subplot(gs_def)

# Create an ffmpeg writer to generate the video
video_filename = 'pitch_ctrl.mp4'
ffmpeg_writer = anim.writers['ffmpeg']
writer = ffmpeg_writer(fps=20)
writer.setup(fig=fig, outfile=video_filename, dpi=100)
for i, frame in enumerate(frame_range):
  # Need to call a custom revision to Metrica_Viz.plot_pitch (hidden in the cell
  # above) which takes in the figure and axis we've already created above.
  plot_pitch2(figax=(fig, ax_pitch), field_color='white',
              field_dimen=field_dimen)
  mviz.plot_frame(home_team.loc[frame],
                  away_team.loc[frame],
                  figax=(fig, ax_pitch),
                  include_player_velocities=True,
                  annotate=True)
  # Look up the attacking team pitch control for this frame and plot it.
  team_pc = player_pitch_control['Team'][i][0]
  ax_pitch.imshow(np.flipud(team_pc),
                  extent=(-field_dimen[0]/2.0, field_dimen[0]/2.0,
                          -field_dimen[1]/2.0, field_dimen[1]/2.0),
                  interpolation='hanning',
                  vmin=0.0,
                  vmax=1.0,
                  cmap='bwr',
                  alpha=0.5)
  # Calculate the continuous average pitch control, for all frames up to the
  # current frame, for both teams.
  att_team_culm_avg = np.mean(
      [ppc[0] for ppc in player_pitch_control['Team'][:i+1]])
  def_team_culm_avg = np.mean(
      [ppc[1] for ppc in player_pitch_control['Team'][:i+1]])
  # Get the attacking and defending player names to use as subplot axis labels.
  att_x_axis = def_x_axis = []
  for k in player_pitch_control.keys():
    if k.startswith('att_'):
      att_x_axis.append(k[4:])
    elif k.startswith('def_'):
      def_x_axis.append(k[4:])
  # Sort by player name length so that 'Player_2' will come before 'Player_10'
  att_x_axis = sorted(att_x_axis, key=lambda x: (len(x), x))
  def_x_axis = sorted(def_x_axis, key=lambda x: (len(x), x))
  # Calculate the continuous average pitch control, for all frames up to and
  # including the current frame, for each attacking player, and subtract the
  # player's "non-contribution" scenario from the team average.
  y_axis = []
  for player_id in att_x_axis:
    player_culm_avg = np.mean(
        [ppc[0] for ppc in player_pitch_control['att_' + player_id][:i+1]])
    y_axis.append(att_team_culm_avg - player_culm_avg)
  att_bars = ax_att.barh(att_x_axis, y_axis, align='center')
  # Do the same for the defending team.
  y_axis = []
  for player_id in def_x_axis:
    player_culm_avg = np.mean(
        [ppc[1] for ppc in player_pitch_control['def_' + player_id][:i+1]])
    y_axis.append(def_team_culm_avg - player_culm_avg)
  def_bars = ax_def.barh(def_x_axis, y_axis, align='center')
  for bar in att_bars + def_bars:
    if bar.get_width() < 0:
      bar.set_color('r')
    else:
      bar.set_color('g')
  # Axis settings need to be set every loop since
  # they are cleared at the end of the loop.
  ax_att.set_xlim([-0.012, 0.012])
  ax_att.set_title('Attacking team')
  ax_def.set_xlim([-0.025, 0.025])
  ax_def.yaxis.set_label_position('right')
  ax_def.yaxis.tick_right()
  ax_def.set_title('Defending team')
  # Grab frame.
  writer.grab_frame()
  # Clear all axis.
  ax_att.cla()
  ax_pitch.cla()
  ax_def.cla()
writer.finish()

In [ ]:
# Run this cell to display the video.
from ipywidgets import Video
Video.from_file(video_filename)